# MCP

可以将 MCP 想象成 AI 应用程序的"USB-C端口"，提供了一种将AI模型连接到不同数据源和工具的标准化方法。

传统上，M个AI应用程序连接到N个工具/服务需要 M×N 个自定义集成。而MCP能将复杂的 M×N 问题转化为更易管理的 M+N 问题，每个主机和服务器只需要实现一次MCP标准

## MCP 架构组件
### 主机 (Host)
定义：终端用户直接交互的面向用户的AI应用程序
举例：
- Anthropic的Claude Desktop
- Cursor等AI增强的IDE

### 客户端 (Client)
定义：主机应用程序中管理与特定MCP服务器通信的组件
举例：在Claude Desktop中，负责与MCP服务器交互的模块，每个客户端与单个服务器保持1:1连接

### 服务器 (Server)
定义：通过MCP协议公开功能（工具、资源、提示）的外部程序或服务
举例：
- 文件系统访问服务器（提供文件读写功能）
- 数据库查询服务器（提供SQL查询功能）
- 12306车票查询服务器（提供火车票查询功能）

![image.png](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*LtZYdQvtDLzN4CwmlHvfFA.png)

## 传输方式

传输方式是 MCP 协议中定义的客户端和服务器之间通信的具体实现方式。

所有传输方式都使用相同的 JSON-RPC 消息格式，但它们在如何传递这些消息上有所不同。

![](https://raw.githubusercontent.com/InTheFuture7/attachment/main/202507222318840.png)

## MCP实现

![](https://raw.githubusercontent.com/InTheFuture7/attachment/main/202507222314711.png)

## 第三方MCP调用

魔搭MCP市场上的MCP服务分为两类，带**Hosted标签**的为**在线托管服务**，用户可以直接使用；带**Local标签**的为**本地部署服务**，用户需要自行部署。

12306-MCP 为例说明如何使用魔搭MCP市场的在线托管服务。

In [ ]:
import asyncio
from agents import Agent, Runner, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
from agents.mcp.server import MCPServerSse
from agents.model_settings import ModelSettings
import os
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
chat_model = "mistral-small-latest"
base_url="https://api.mistral.ai/v1"
api_key=os.getenv('mistral_key')
mcp_url = os.getenv('mcp_url')

set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

async def main():
    async with MCPServerSse(
        params={
            "url": mcp_url,
        }
    ) as my_mcp_server: # 建议用这种上下文写法，否则需要手动连接和关闭MCP服务。
        agent = Agent(
            name="Assistant",
            instructions="你是一个火车票查询助手，能够查询火车票信息。",
            mcp_servers=[my_mcp_server],
            model_settings=ModelSettings(tool_choice="required"),
            model=llm,
        )

        message = "明天从[]]到[]可以买哪些火车票？"
        print(f"Running: {message}")
        result = await Runner.run(starting_agent=agent, input=message)
        print(result.final_output)


# if __name__ == "__main__":
#     asyncio.run(main())

await main()